In [1]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random

# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential, Model
# from keras.layers import (
#     Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,
#     GlobalAveragePooling2D
# )
# from keras.optimizers import Adam
# from keras.applications import ResNet50
# from keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,
    GlobalAveragePooling2D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical





In [2]:
def load_labels(base_dir, num_samples=10):
    txt_data = {}
    label_files = []
    
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".txt"):
                label_files.append(os.path.join(root, file))
    
    selected_label_files = random.sample(label_files, min(num_samples, len(label_files)))
    
    for file_path in selected_label_files:
        try:
            with open(file_path, 'r') as f:
                txt_data[file_path] = f.read()
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    print(f"Loaded {len(txt_data)} TXT files from {base_dir}")
    return txt_data

def load_images(base_dir, num_samples=10):
    images = []
    filenames = []
    image_files = []
    
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_files.append(os.path.join(root, file))
    
    selected_image_files = random.sample(image_files, min(num_samples, len(image_files)))
    
    for file_path in selected_image_files:
        try:
            image = Image.open(file_path).resize((224, 224)).convert('RGB')
            images.append(np.array(image))
            filenames.append(file_path)
        except UnidentifiedImageError as e:
            print(f"Error loading image {file_path}: {e}")
    
    print(f"Loaded {len(images)} images from {base_dir}")
    return np.array(images), filenames


In [3]:
BASE = "Database_Realtime"
IMU = os.path.join(BASE, "IMU")
VISION = os.path.join(BASE, "Vision/Webcam")

print("Loading Labels...")
txt_files = load_labels(IMU, num_samples=10)

print("Loading Images...")
images, image_paths = load_images(VISION, num_samples=10)

print("Image shape:", images.shape)
print("Labels:", len(txt_files))


Loading Labels...
Loaded 10 TXT files from Database_Realtime\IMU
Loading Images...
Loaded 10 images from Database_Realtime\Vision/Webcam
Image shape: (10, 224, 224, 3)
Labels: 10


In [4]:
def parse_labels(txt_data):
    labels = []
    for file_path, data in txt_data.items():
        lines = data.splitlines()
        for line in lines:
            values = line.split()
            if len(values) >= 4:
                pose = int(values[0]) 
                depth = float(values[1])  
                pitch = float(values[2])  
                roll = float(values[3])  
                labels.append([pose, depth, pitch, roll])
    return np.array(labels)

y_labels = parse_labels(txt_files)
print("Parsed Labels Shape:", y_labels.shape)


Parsed Labels Shape: (1619, 4)


In [13]:


# Normalize images and split data
X_images = images / 255.0  # Normalize
y_pose = y_labels[:, 0]
depth_labels = y_labels[:, 1]

# Normalize depth labels
depth_labels = depth_labels[:X_images.shape[0]]
y_labels = np.stack((y_pose, depth_labels), axis=1)


# Split data
X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



ValueError: all input arrays must have the same shape

In [6]:
def create_cnn_model(input_shape, num_classes=5):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_cnn_model(X_train.shape[1:], num_classes=5)
model.summary()


NameError: name 'X_train' is not defined

In [7]:
y_train_classes = y_train[:, 0].astype('int') 
y_val_classes = y_val[:, 0].astype('int')

history_cnn = model.fit(
    X_train, y_train_classes,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val_classes),
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, y_test[:, 0].astype('int'), verbose=2)
print(f"Test accuracy (CNN): {test_acc}")


NameError: name 'y_train' is not defined

In [8]:
def create_mlp(input_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

mlp_model = create_mlp((X_train.shape[1] * X_train.shape[2] * X_train.shape[3],))
history_mlp = mlp_model.fit(
    X_train.reshape(X_train.shape[0], -1), y_train[:, 1],  
    epochs=20,
    batch_size=32,
    validation_data=(X_val.reshape(X_val.shape[0], -1), y_val[:, 1]),
    verbose=1
)

test_loss_mlp, test_mae_mlp = mlp_model.evaluate(X_test.reshape(X_test.shape[0], -1), y_test[:, 1], verbose=2)
print(f"Test MAE (MLP): {test_mae_mlp}")


NameError: name 'X_train' is not defined

In [9]:
def create_resnet(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(1)(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

resnet_model = create_resnet(X_train.shape[1:])
history_resnet = resnet_model.fit(
    X_train, y_train[:, 1],
    epochs=20,
    batch_size=16,
    validation_data=(X_val, y_val[:, 1]),
    verbose=1
)

test_loss_resnet, test_mae_resnet = resnet_model.evaluate(X_test, y_test[:, 1], verbose=2)
print(f"Test MAE (ResNet): {test_mae_resnet}")


NameError: name 'X_train' is not defined

In [10]:
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return np.sqrt(mse)

y_pred = resnet_model.predict(X_test)
rmse = calculate_rmse(y_test[:, 1], y_pred)
print(f"RMSE: {rmse}")


NameError: name 'resnet_model' is not defined

In [11]:
def plot_history(history, model_name):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

# Example
# plot_history(history_cnn, "CNN")
